# Real-time Inference with Jina Reranking Model Package

This notebook shows you how to deploy the following reranking models using Amazon SageMaker and perform inference with it:
 - [jina-reranker-v2-base-multilingual](https://aws.amazon.com/marketplace/pp/prodview-5jrm2ulfix372?sr=0-14&ref_=beagle&applicationId=AWSMPContessa) 
 - [jina-reranker-v1-base-en](https://aws.amazon.com/marketplace/pp/prodview-avmxk2wxbygd6?sr=0-12&ref_=beagle&applicationId=AWSMPContessa)  
 - [jina-reranker-v1-turbo-en](https://aws.amazon.com/marketplace/pp/prodview-hayk4sjrbyd5g?sr=0-13&ref_=beagle&applicationId=AWSMPContessa) 
 - [jina-reranker-v1-tiny-en](https://aws.amazon.com/marketplace/pp/prodview-5jrm2ulfix372?sr=0-14&ref_=beagle&applicationId=AWSMPContessa) 

## Pre-requisites:
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to above model(s).

## Contents:
1. [Subscribe to the model package](#1.-Model-package-setup)
2. [Real-time inference](#2.-Real-time-inference)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))

# 1. Model package setup

Please subscribe to the model package from AWS Marketplace.

Install `jina-sagemaker` package 


```bash
pip install --upgrade jina-sagemaker
```

In [ ]:
# Specify the role as required by SageMaker
role = "..."

In [ ]:
import boto3

region = boto3.Session().region_name

# Mapping for Model Package Names
model_name_map = {
    "jina-reranker-v2-base-multilingual": "tbd",
    "jina-reranker-v1-base-en": "jina-reranker-v1-base-en-77e50152c042315da374fb388ad6f40d",
    "jina-reranker-v1-turbo-en": "jina-reranker-v1-turbo-en-643a1c3bd23a3b298cb4b6caf43eaf91",
    "jina-reranker-v1-tiny-en": "jina-reranker-v1-tiny-en-209f8205ecad33c1a504ac85e9b79a53",
}

# Specify the model name, jina-reranker-v1-base-en is picked here for example
model_name = model_name_map["jina-reranker-v1-base-en"]

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{model_name}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{model_name}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{model_name}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{model_name}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{model_name}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{model_name}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{model_name}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{model_name}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{model_name}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{model_name}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{model_name}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{model_name}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{model_name}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{model_name}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{model_name}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{model_name}",
}

# Specify the model you want to use
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]

---

# 2. Real-time inference

To learn about real-time inference capabilities in Amazon SageMaker, please refer to the [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html).

Let's create an endpoint with static instances.

In [ ]:
from jina_sagemaker import Client

client = Client(region_name=region)
endpoint_name = "my-endpoint"

We can create a new endpoint using the `create_endpoint` method and passing the required parameters like `instance_type`, `n_instances` etc.

In [ ]:
client.create_endpoint(
    arn=model_package_arn, 
    role=role, 
    endpoint_name=endpoint_name, 
    instance_type="ml.g5.xlarge", 
    n_instances=1,
)

Or, we can connect to an existing endpoint using the `connect_to_endpoint` method by passing the endpoint name.

In [ ]:
client.connect_to_endpoint(endpoint_name=endpoint_name)

## Perform real-time inference

### Usage with `jina-sagemaker` sdk

In [ ]:
result = client.rerank(
    documents=["the dog is in my house", "he likes dog", "hello world"],
    query="where is the dog",
    top_n=2,
)

print(result)

### Usage with aws-sdk

Create a input file `input.json` with the following content.

```json
{
    "data": {
        "documents": [{"text": "the dog is in my house"},
                      {"text": "he likes dog"},
                      {"text": "hello world"}],
        "query": "where is the dog",
        "top_n": 2
    }
}
```

Run the AWS `invoke-endpoint` CLI.

In [ ]:
aws sagemaker-runtime invoke-endpoint \
--endpoint-name <endpoint-name> \
--content-type 'application/json' \
--body fileb://input.json \
output.json

---

# 3. Clean-up

## A. Delete the model

In [ ]:
client.delete_endpoint()
client.close()

## B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.
